# 测试残差正交的效果

结论：
1. 对alpha进行残差正交并不会影响alpha因子的回归系数
2. 对alpha进行残差会影响risk因子的回归系数

In [2]:
import pandas as pd
import os
import numpy as np
#import tushare as ts

os.chdir("E:\\python\\project\\Cubicial_Quant")

# 先组合因子，再产生y

In [6]:
x = (np.random.random(10000)* 10).reshape(2000,5)
x

array([[0.95992644, 0.29740538, 8.07207903, 4.26517082, 9.8688344 ],
       [7.66399246, 4.56690374, 4.92253012, 7.32737101, 3.85210607],
       [2.63619461, 9.0023287 , 6.87709723, 0.42257403, 2.15728482],
       ...,
       [1.8556493 , 5.76815066, 8.13504373, 2.13188112, 7.63904623],
       [6.61517947, 8.7481047 , 3.75389053, 0.25687306, 9.88180845],
       [3.75154625, 0.14699595, 9.66317253, 1.07223207, 9.20615716]])

In [7]:
x_mat = np.ones(20000).reshape(2000,10)

In [8]:
x_mat[:,:5] = x

In [24]:
np.random.seed(42)
x_mat[:,6] = x_mat[:,[4,1]].dot(np.array([0.5,0.6])) + np.random.rand(2000) + 1
x_mat[:,7] = x_mat[:,[2,3]].dot(np.array([0.5,0.6])) + np.random.rand(2000) + 1
x_mat[:,8] = x_mat[:,[2,4]].dot(np.array([0.5,0.6])) + np.random.rand(2000) + 1
x_mat[:,9] = x_mat[:,[1,3]].dot(np.array([0.5,0.6])) + np.random.rand(2000) + 1
x_mat[:,5] = x_mat[:,[4,3]].dot(np.array([0.5,0.6])) + np.random.rand(2000) + 1

In [25]:
import statsmodels.api as sm

In [26]:
x_mat_cons = sm.add_constant(x_mat)

In [27]:
y = x_mat_cons.dot(np.array([0.6,0.7,0.8,0.9,1.1,1.2,1.3,1.4,1.5,1.6,1.7])) + np.random.rand(2000) 

In [28]:
coef_cons_uno = sm.OLS(y,x_mat_cons).fit().params
coef_cons_uno

array([1.09738762, 0.69814565, 0.7747947 , 0.9067373 , 1.08306914,
       1.24543534, 1.25697594, 1.40581637, 1.52935201, 1.55766761,
       1.74548639])

In [29]:
for i in range(5,10):
    coef = sm.OLS(x_mat[:,i],x_mat_cons[:,:6]).fit().params
    residue = x_mat[:,i] - x_mat_cons[:,:6].dot(coef)
    x_mat[:,i] = residue

In [30]:
x_mat_cons = sm.add_constant(x_mat)
coef_cons_or = sm.OLS(y,x_mat_cons).fit().params
coef_cons_or

array([12.35379286,  0.69845235,  2.49772236,  2.44259218,  3.79673807,
        3.5058327 ,  1.25697594,  1.40581637,  1.52935201,  1.55766761,
        1.74548639])

正交因子的系数都没有变化，作为基准的因子的系数增大

# 改变y的组合方式，先产生y，再组合因子

In [46]:
for i in range(5,10):
    x_mat[:,i] = np.random.rand(2000)
x_mat_cons = sm.add_constant(x_mat)

# 产生y
y = x_mat_cons.dot(np.array([0.6,0.7,0.8,0.9,1.1,1.2,1.3,1.4,1.5,1.6,1.7]))   

#组合因子
x_mat[:,6] = x_mat[:,6]+ x_mat[:,:5].dot(np.array([0.1]*5))+ 0.1
x_mat[:,7] = x_mat[:,7]+ x_mat[:,:5].dot(np.array([0.15]*5))+ 0.1
x_mat[:,8] = x_mat[:,8]+ x_mat[:,:5].dot(np.array([0.2]*5))+ 0.1
x_mat[:,9] = x_mat[:,9]+ x_mat[:,:5].dot(np.array([0.25]*5))+ 0.1
x_mat[:,5] = x_mat[:,5]+ x_mat[:,:5].dot(np.array([0.3]*5))+ 0.1
x_mat_cons = sm.add_constant(x_mat)

In [47]:
coef_cons_uno = sm.OLS(y,x_mat_cons).fit().params
coef_cons_uno

array([-0.15, -0.8 , -0.7 , -0.6 , -0.4 , -0.3 ,  1.3 ,  1.4 ,  1.5 ,
        1.6 ,  1.7 ])

In [48]:
# 正交化
for i in range(5,10):
    coef = sm.OLS(x_mat[:,i],x_mat_cons[:,:6]).fit().params
    residue = x_mat[:,i] - x_mat_cons[:,:6].dot(coef)
    x_mat[:,i] = residue

In [49]:
x_mat_cons = sm.add_constant(x_mat)
coef_cons_or = sm.OLS(y,x_mat_cons).fit().params
coef_cons_or

array([4.32296524, 0.6878245 , 0.80856167, 0.90481132, 1.09904009,
       1.20292842, 1.3       , 1.4       , 1.5       , 1.6       ,
       1.7       ])

In [50]:
sum(x_mat)

array([ 9.84000981e+03,  1.00851300e+04,  1.00262423e+04,  9.96850975e+03,
        1.01036118e+04, -2.23909780e-12, -1.04183329e-12, -2.54063437e-12,
       -1.78346227e-12, -5.51469981e-12])

In [51]:
np.random.rand(2000)

array([0.00756341, 0.66353845, 0.91346002, ..., 0.70891607, 0.75669601,
       0.61296342])

# 实例测试

In [67]:
# 阅读市值作为因子
factor = pd.read_csv("./alpha/alpha10.csv",index_col = 0)

In [68]:
re = pd.read_csv("./data/Return.csv",index_col = 0)

In [69]:
re = re.reindex(index = factor.index,columns = factor.columns)

In [78]:
x = factor.iloc[:,-10:]
y = re.iloc[:,-1]

In [79]:
factor

,20000110,20000111,20000112,20000113,20000114,20000117,20000118,20000119,20000120,20000121,...,20181211,20181212,20181213,20181214,20181217,20181218,20181219,20181220,20181221,20181224
S_INFO_WINDCODE,,,,,,,,,,,,,,,,,,,,,
000001.SZ,14.955079,14.896810,14.855988,14.850491,14.836054,14.847178,14.825371,14.831008,14.832131,14.840518,...,16.681438,16.682415,16.696957,16.675555,16.687286,16.670627,16.652680,16.629269,16.602128,16.598948
000002.SZ,13.344058,13.290185,13.254500,13.252586,13.212522,13.244894,13.209527,13.219477,13.237142,13.217495,...,17.171316,17.185842,17.213907,17.190008,17.192273,17.153052,17.148335,17.145178,17.102790,17.087415
000003.SZ,12.264642,12.191260,12.187870,12.222903,12.167286,12.186171,12.170746,12.169018,12.155080,12.148038,...,12.245594,12.245594,12.245594,12.245594,12.245594,12.245594,12.245594,12.245594,12.245594,12.245594
000004.SZ,11.373662,11.375590,11.324196,11.307835,11.337294,11.386132,11.434513,11.451623,11.500413,11.549365,...,11.857088,11.843303,11.871860,11.940422,11.901333,11.863615,11.834818,11.835427,11.825032,11.836035
000005.SZ,12.802517,12.766202,12.722355,12.719259,12.736170,12.832034,12.853953,12.838936,12.895100,12.885956,...,12.681671,12.671753,12.668425,12.648223,12.651618,12.634524,12.624125,12.620635,12.610090,12.613617
000006.SZ,12.522292,12.457383,12.423278,12.416109,12.389024,12.402659,12.387968,12.389024,12.407853,12.398483,...,13.593051,13.581176,13.577757,13.556995,13.570884,13.512305,13.515955,13.508642,13.497572,13.486378
000007.SZ,11.834962,11.769382,11.729717,11.725082,11.683582,11.702768,11.720425,11.746909,11.744634,11.771602,...,12.571565,12.564328,12.597664,12.544767,12.603502,12.669930,12.631058,12.619668,12.577557,12.597664
000008.SZ,12.864779,12.879911,12.906219,12.949517,12.992697,13.047831,13.112211,13.177586,13.228671,13.291303,...,13.902324,13.904905,13.960057,13.927836,13.912607,13.904905,13.889320,13.891935,13.897143,13.925314
000009.SZ,12.821530,12.787271,12.771055,12.771055,12.779196,12.762847,12.768326,12.760096,12.757337,12.746226,...,13.825651,13.838336,13.850863,13.804145,13.817104,13.812803,13.795411,13.804145,13.784386,13.797602


In [80]:
y

S_INFO_WINDCODE
000001.SZ   -0.003174
000002.SZ   -0.015259
000003.SZ         NaN
000004.SZ    0.011041
000005.SZ    0.003431
000006.SZ   -0.011112
000007.SZ    0.020338
000008.SZ    0.028568
000009.SZ    0.013249
000010.SZ   -0.002942
000011.SZ    0.000000
000012.SZ    0.004852
000013.SZ         NaN
000014.SZ   -0.012188
000015.SZ         NaN
000016.SZ   -0.006098
000017.SZ   -0.012298
000018.SZ   -0.029523
000019.SZ    0.000000
000020.SZ    0.011506
000021.SZ    0.001695
000022.SZ         NaN
000023.SZ   -0.016478
000024.SZ         NaN
000025.SZ    0.004752
000026.SZ   -0.001276
000027.SZ    0.001953
000028.SZ    0.010184
000029.SZ         NaN
000030.SZ    0.023868
               ...   
603937.SH    0.024201
603938.SH    0.035403
603939.SH    0.048619
603955.SH   -0.005522
603958.SH    0.014563
603959.SH    0.013477
603960.SH    0.006685
603963.SH    0.026576
603966.SH   -0.012146
603968.SH    0.005874
603969.SH    0.016642
603970.SH    0.006094
603976.SH    0.035860
603977.SH    0.0

In [81]:
y = y.dropna()

In [82]:
x = x.reindex(index = y.index)

In [84]:
x = x.dropna(how = "any",axis = 1)
y = y.reindex(index=  x.index)

In [85]:
x_mat = x.values
y_values = y.values

In [86]:
x.shape,y.shape

((3356, 10), (3356,))

In [87]:
x_mat_cons  = sm.add_constant(x_mat)
coef_unor = sm.OLS(y_values,x_mat_cons).fit().params
coef_unor

array([ 4.02011244e-04,  3.21982331e-03, -5.01696440e-03,  1.05905203e-03,
       -1.65785495e-03,  1.66259648e-03, -1.59842493e-03,  9.32078771e-05,
        2.37634540e-03, -1.00448318e+00,  1.00433124e+00])

In [88]:
# 对后5列残差正交化
for i in range(5,10):
    coef = sm.OLS(x_mat[:,i],x_mat_cons[:,:6]).fit().params
    residue = x_mat[:,i] - x_mat_cons[:,:6].dot(coef)
    x_mat[:,i] = residue
x_mat_cons = sm.add_constant(x_mat)

In [89]:
coef_or = sm.OLS(y_values,x_mat_cons).fit().params
coef_or

array([ 2.75052605e-02,  8.26397420e-02, -2.11762731e-01,  1.35508321e-01,
        6.76019194e-02, -7.54856942e-02, -1.59842493e-03,  9.32078771e-05,
        2.37634540e-03, -1.00448318e+00,  1.00433124e+00])

正交因子的回归系数依然不变

In [90]:
coef_or == coef_unor

array([False, False, False, False, False, False, False, False, False,
       False, False])